In [85]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from random import randint
from sklearn.calibration import LabelEncoder
from common import get_full_data

def heart_failure():
    df = pd.read_csv("dataset/heart_failure_clinical_records_dataset.csv")
    # get dependent and independent features
    X=df.iloc[:,:-1]
    y=df.iloc[:,-1]
    classes = y.unique()
    X,y=get_full_data(X,y)
    return X,y,[str(c) for c in classes]

def encode_categorical_columns(df):
    le = LabelEncoder()
    # Loop over all columns in the DataFrame
    for col in df.columns:
        # Check if the column is of type object (string)
        if df[col].dtype == 'object':
            # Use LabelEncoder to do the numeric transformation
            df[col] = le.fit_transform(df[col])
    return df

def disease():
    df = pd.read_csv("dataset/disease.csv")
    # get dependent and independent features
    symptoms = df.iloc[:,1:]
    all_symptoms = set()
    for col in symptoms.columns:
        unq = symptoms[col].unique()
        unq=[str.strip(v) for v in unq if isinstance(v,str)]
        all_symptoms.update(unq)
    
    all_symptoms=list(all_symptoms)
    symptoms_embedding = pd.DataFrame(
        np.zeros((len(df),len(all_symptoms))),
        columns=all_symptoms
    )
    
    uniques = symptoms.apply(lambda x: np.unique(np.array(x.dropna(),dtype=str)),axis=1)
    for id,u in enumerate(uniques):
        u=[str.strip(v) for v in u if isinstance(v,str)]
        symptoms_embedding.iloc[id][u]=1

    y = df.iloc[:,0]
    classes = y.unique()
    
    for index,cls in enumerate(classes):
        y[y==cls]=index
    
    return symptoms_embedding, y.astype(int), classes

def AIDS():
    df = pd.read_csv("dataset/AIDS_Classification.csv")
    X=df.iloc[:,:-1]
    y=df.iloc[:,-1]
    classes = y.unique()
    X,y=get_full_data(X,y)
    return X,y,[str(c) for c in classes]

def seeds():
    df = pd.read_csv("dataset/seeds.csv")
    y = df.iloc[:,-1]-1
    classes = y.apply(lambda x: str(x)).unique()
    X = df.iloc[:,:-1]
    return X,y, classes

def housing():
    df = pd.read_csv("dataset/housing.csv")
    y=df['SaleCondition']
    bad_classes_names=['Alloca','AdjLand','Normal','Partial']
    classes = set(y.apply(lambda x: str(x)).unique())-set(bad_classes_names)

    X=df.drop(columns=['SaleCondition','SalePrice'])
    bad_classes = np.any([y == cl for cl in bad_classes_names],axis=0)
    X=encode_categorical_columns(X)

    X=np.array(X)
    y=np.array(y)

    y=y[~bad_classes]
    X=X[~bad_classes]


    y_encoded = np.zeros_like(y,dtype=np.int32)
    for i,cls in enumerate(classes):
        y_encoded[y==cls]=i

    return X,y_encoded,classes

In [86]:
from xgboost import XGBClassifier
# X,y, y_classes = seeds()
X,y, y_classes = heart_failure()

# for high-dimensional data use `gpu` for device if you have one
special_model = XGBClassifier(device='cpu')

In [87]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from common import XGB_search_params

params = XGB_search_params()
state = randint(0,1000)
search = RandomizedSearchCV(
    special_model,
    params,
    n_iter=200,
    cv=5,
    random_state=state,
    n_jobs=-1,
)

search.fit(X,y)
special_model=search.best_estimator_

In [88]:
# do repeated stratified k-fold cross-validation with classification report
from sklearn.model_selection import RepeatedStratifiedKFold
from common import cross_val_classification_report

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=50)
report = cross_val_classification_report(
    model=special_model,
    X=np.array(X),
    y=y,
    cv=cv,
    target_names=y_classes
)
print(report)

              precision    recall  f1-score   support

           1       0.84      0.94      0.89       609
           0       0.84      0.62      0.71       288

    accuracy                           0.84       897
   macro avg       0.84      0.78      0.80       897
weighted avg       0.84      0.84      0.83       897



In [89]:
# New method
from common import find_outliers

X_numpy = np.array(X)
y_numpy = np.array(y)

outliers_to_remove=0.3

outliers_mask, score = find_outliers(
    X_numpy,
    y_numpy,
    special_model,
    outliers_to_remove=outliers_to_remove,
    iterations=5,
    gamma=0.9,
    evaluate_loss=metrics.mean_absolute_error,
    cv=5,
    repeats=5,
    class_weight_scale_power=0.5,
    plot=False
)

X_clean = X_numpy[~outliers_mask]
y_clean = y_numpy[~outliers_mask]
removed_size = np.count_nonzero(outliers_mask)/len(outliers_mask)

print("removed ",removed_size)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=50)
report = cross_val_classification_report(
    model=special_model,
    X=X_clean,
    y=y_clean,
    cv=cv,
    target_names=y_classes
)
print(report)


removed  0.19063545150501673
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       537
           0       1.00      1.00      1.00       189

    accuracy                           1.00       726
   macro avg       1.00      1.00      1.00       726
weighted avg       1.00      1.00      1.00       726



In [101]:
from common import generate_colors_for_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import KernelPCA as KPCA
from kernel_pca_search import KernelPCASearchCV, kernel_pca_scorer
from render import *

scaler = RobustScaler()
dot_size = 8
X_numpy[np.isnan(X_numpy)]=-1
X_ = scaler.fit_transform(X_numpy)

transform = KernelPCASearchCV(n_components=2,kernel='rbf')
transform.fit(X_)

def render(model,X_clean,y_clean,title="data plot"):
    colors = generate_colors_for_classification(y_clean,seed=100)
    data = np.concatenate([X_clean,colors],axis=1)
    removed_size = 1-len(y_clean)/len(X)

    print("removed ",removed_size)

    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=50)
    report = cross_val_classification_report(
        model=model,
        X=X_clean,
        y=y_clean,
        cv=cv,
        target_names=y_classes
    )
    print(report)
    plot_2d_rgb(data,title,["d1","d2","d3"],dot_size)

render(special_model,transform.transform(X_),y,"original data")

render(special_model,transform.transform(X_)[~outliers_mask],y.to_numpy()[~outliers_mask],"cleaned data")
# print(kernel_pca_scorer(transform,X_))

removed  0.0
              precision    recall  f1-score   support

           1       0.72      0.92      0.81       609
           0       0.61      0.25      0.36       288

    accuracy                           0.71       897
   macro avg       0.67      0.59      0.58       897
weighted avg       0.69      0.71      0.67       897



removed  0.1906354515050167
              precision    recall  f1-score   support

           1       0.78      0.98      0.87       537
           0       0.77      0.23      0.35       189

    accuracy                           0.78       726
   macro avg       0.77      0.60      0.61       726
weighted avg       0.78      0.78      0.73       726



In [102]:
data = np.concatenate([X_,y.to_numpy()[:,np.newaxis]],axis=1)

In [103]:
from sklearn.svm import OneClassSVM

def remove_outliers_svm(X):
    svm = OneClassSVM(nu=removed_size, kernel="rbf", gamma=0.2)
    y_pred = svm.fit_predict(X)
    return X[y_pred == 1]  # Retain only inliers

svm_data_clean = remove_outliers_svm(data)
X_clean=svm_data_clean[:,:-1]
y_clean=svm_data_clean[:,-1]

render(special_model,transform.transform(X_clean),y_clean,"svm one class")

removed  0.22742474916387956
              precision    recall  f1-score   support

           1       0.72      0.99      0.83       495
           0       0.45      0.03      0.05       198

    accuracy                           0.71       693
   macro avg       0.59      0.51      0.44       693
weighted avg       0.64      0.71      0.61       693



In [104]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(random_state=50,contamination=removed_size)
outliers_pred=clf.fit_predict(data)

data_clean = data[outliers_pred==1]

X_clean=data_clean[:,:-1]
y_clean=data_clean[:,-1]

render(special_model,transform.transform(X_clean),y_clean,"isolation forest")

removed  0.1906354515050167
              precision    recall  f1-score   support

           1       0.79      1.00      0.88       570
           0       0.00      0.00      0.00       156

    accuracy                           0.79       726
   macro avg       0.39      0.50      0.44       726
weighted avg       0.62      0.79      0.69       726



/home/vlad/Programs/Git/python-outliers-detector/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/vlad/Programs/Git/python-outliers-detector/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/vlad/Programs/Git/python-outliers-detector/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

